Import Libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import folium #import mapping

Import Census Data

In [2]:
dfCensus = pd.read_csv("sub-est.csv")
dfCensus.head()

,City,Population
0,New York City,8230290
1,Buffalo,254290
2,Rochester,205077
3,Yonkers,201344
4,Syracuse,141491


Import Names of Town of Oyster Bay Cities

In [3]:
dfVillages = pd.read_csv("Villages.csv", names = ["City"])
dfVillages.head()

,City
0,Amityville
1,Bayville
2,Brookville
3,Centre Island
4,Cove Neck


FOURSQUARE DATA

In [4]:
CLIENT_ID = '' 
CLIENT_SECRET = '' 
ACCESS_TOKEN = '' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
radius = 0

In [5]:
address = 'Copiague, NY'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
search_query = 'Pizza'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
# assign relevant part of JSON to venues
venues = results['response']['venues']
df = json_normalize(venues)

<ipython-input-5-7ab06bdf9c42>:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(venues)


Loop Through Villages and Add to Data Frame

In [6]:
for ind in dfVillages['City']:
    radius = 10000
    address = ind + ', NY'
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    search_query = 'Pizza'
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    # assign relevant part of JSON to venues
    venues = results['response']['venues']
    df = df.append(json_normalize(venues), ignore_index='True')

<ipython-input-6-8f633ba26ecd>:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = df.append(json_normalize(venues), ignore_index='True')


In [7]:
df.shape

(1080, 24)

In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
df_filtered = df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_filtered['categories'] = df_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_filtered.columns = [column.split('.')[-1] for column in df_filtered.columns]

df_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,A Touch of Italy Brick Oven Pizza,Pizza Place,355 Broadway,40.684060,-73.417533,"[{'label': 'display', 'lat': 40.68405999999999...",565,11701,US,Amityville,NY,United States,"[355 Broadway, Amityville, NY 11701, United St...",NaN,4d86a24b02eb5481283d5cf5
1,Raimo's Wood Fired Brick Oven Pizza,Pizza Place,45 Merrick Rd,40.671438,-73.421956,"[{'label': 'display', 'lat': 40.6714375170934,...",936,11701,US,Amityville,NY,United States,"[45 Merrick Rd, Amityville, NY 11701, United S...",NaN,4ba3e563f964a520326b38e3
2,Fontana's Pizza,Pizza Place,217 Broadway,40.678711,-73.417007,"[{'label': 'display', 'lat': 40.6787109375, 'l...",31,11701,US,Amityville,NY,United States,"[217 Broadway, Amityville, NY 11701, United St...",NaN,4f90a180e4b042e66189b669
3,Mikes Pizza,Pizza Place,217 Broadway,40.678711,-73.417007,"[{'label': 'display', 'lat': 40.6787109375, 'l...",31,11701,US,Amityville,NY,United States,"[217 Broadway (btw Oak and Union), Amityville,...",btw Oak and Union,4bff67f788e795213df67d8f
4,Carolina's Ii Pizza & Restaurant,Pizza Place,59 Merrick Rd,40.671272,-73.420975,"[{'label': 'display', 'lat': 40.67127227783203...",920,11701,US,Amityville,NY,United States,"[59 Merrick Rd, Amityville, NY 11701, United S...",NaN,4e4e4fd0bd4101d0d7a7728b


REMOVE COLUMNS

In [9]:
df_filtered.drop(['labeledLatLngs', 'cc', 'country', 'formattedAddress', 'crossStreet', 'distance', 'address'], axis=1, inplace=True)
df_filtered.head()

,name,categories,lat,lng,postalCode,city,state,id
0,A Touch of Italy Brick Oven Pizza,Pizza Place,40.684060,-73.417533,11701,Amityville,NY,4d86a24b02eb5481283d5cf5
1,Raimo's Wood Fired Brick Oven Pizza,Pizza Place,40.671438,-73.421956,11701,Amityville,NY,4ba3e563f964a520326b38e3
2,Fontana's Pizza,Pizza Place,40.678711,-73.417007,11701,Amityville,NY,4f90a180e4b042e66189b669
3,Mikes Pizza,Pizza Place,40.678711,-73.417007,11701,Amityville,NY,4bff67f788e795213df67d8f
4,Carolina's Ii Pizza & Restaurant,Pizza Place,40.671272,-73.420975,11701,Amityville,NY,4e4e4fd0bd4101d0d7a7728b


In [10]:
#Merge Village Dataframe with Census Dataframe
dfCen = pd.merge(left=dfCensus, right=dfVillages, left_on='City', right_on='City')
#Remove Duplicates
dfCen = dfCen.drop_duplicates(['City'])

In [11]:
dfCen.head()

,City,Population
0,Hicksville,42709
1,Plainview,26453
2,Massapequa,21784
3,Syosset,19453
4,North Massapequa,18505


In [12]:
#Merge datasets
dfDataSet = pd.merge(left=df_filtered,  right=dfCen, left_on='city', right_on='City')

In [13]:
dfDataSet.head()

,name,categories,lat,lng,postalCode,city,state,id,City,Population
0,A Touch of Italy Brick Oven Pizza,Pizza Place,40.684060,-73.417533,11701,Amityville,NY,4d86a24b02eb5481283d5cf5,Amityville,9321
1,Raimo's Wood Fired Brick Oven Pizza,Pizza Place,40.671438,-73.421956,11701,Amityville,NY,4ba3e563f964a520326b38e3,Amityville,9321
2,Fontana's Pizza,Pizza Place,40.678711,-73.417007,11701,Amityville,NY,4f90a180e4b042e66189b669,Amityville,9321
3,Mikes Pizza,Pizza Place,40.678711,-73.417007,11701,Amityville,NY,4bff67f788e795213df67d8f,Amityville,9321
4,Carolina's Ii Pizza & Restaurant,Pizza Place,40.671272,-73.420975,11701,Amityville,NY,4e4e4fd0bd4101d0d7a7728b,Amityville,9321


In [14]:
#Drop and Rename Columns
dfDataSet.drop(['city', 'state', 'id', 'postalCode'], axis=1, inplace=True)
dfDataSet.rename(columns={"name": "Name", "categories": "Categories", "lat": "Lat", "lng": "Lng" }, inplace = 'True')
dfDataSet.head()

,Name,Categories,Lat,Lng,City,Population
0,A Touch of Italy Brick Oven Pizza,Pizza Place,40.684060,-73.417533,Amityville,9321
1,Raimo's Wood Fired Brick Oven Pizza,Pizza Place,40.671438,-73.421956,Amityville,9321
2,Fontana's Pizza,Pizza Place,40.678711,-73.417007,Amityville,9321
3,Mikes Pizza,Pizza Place,40.678711,-73.417007,Amityville,9321
4,Carolina's Ii Pizza & Restaurant,Pizza Place,40.671272,-73.420975,Amityville,9321


In [15]:
#reset index
dfDataSet = dfDataSet.reset_index()
del dfDataSet['index']

In [16]:
dfDataSet.shape

(622, 6)

Group Number of Pizza Shops by City

In [17]:
dfShops = dfDataSet.groupby(['City']).count()
dfShops.rename(columns={"Name" : "PerCity"}, inplace='True')
dfShops.drop(['Categories', 'Lat', 'Lng', 'Population'], axis=1, inplace=True)

In [18]:
df = pd.merge(left=dfDataSet,  right=dfShops, left_on='City', right_on='City')
df.head()

,Name,Categories,Lat,Lng,City,Population,PerCity
0,A Touch of Italy Brick Oven Pizza,Pizza Place,40.684060,-73.417533,Amityville,9321,24
1,Raimo's Wood Fired Brick Oven Pizza,Pizza Place,40.671438,-73.421956,Amityville,9321,24
2,Fontana's Pizza,Pizza Place,40.678711,-73.417007,Amityville,9321,24
3,Mikes Pizza,Pizza Place,40.678711,-73.417007,Amityville,9321,24
4,Carolina's Ii Pizza & Restaurant,Pizza Place,40.671272,-73.420975,Amityville,9321,24


In [19]:
df.loc[:,'Resident_per_shops'] = df['Population']/df['PerCity']

In [20]:
df.head()

,Name,Categories,Lat,Lng,City,Population,PerCity,Resident_per_shops
0,A Touch of Italy Brick Oven Pizza,Pizza Place,40.684060,-73.417533,Amityville,9321,24,388.375
1,Raimo's Wood Fired Brick Oven Pizza,Pizza Place,40.671438,-73.421956,Amityville,9321,24,388.375
2,Fontana's Pizza,Pizza Place,40.678711,-73.417007,Amityville,9321,24,388.375
3,Mikes Pizza,Pizza Place,40.678711,-73.417007,Amityville,9321,24,388.375
4,Carolina's Ii Pizza & Restaurant,Pizza Place,40.671272,-73.420975,Amityville,9321,24,388.375


In [21]:
dfShops = df[['City', 'Resident_per_shops']]
dfShops = dfShops.drop_duplicates(['City', 'Resident_per_shops'])
#reset index
dfShops = dfShops.reset_index()
del dfShops['index']

In [22]:
dfShops.sort_values(by=['Resident_per_shops'], ascending=False)

,City,Resident_per_shops
19,North Massapequa,2313.125000
5,Seaford,1156.923077
2,Massapequa Park,1137.400000
4,South Farmingdale,951.812500
17,Westbury,853.055556
15,Jericho,694.450000
18,Bethpage,629.535714
13,Sea Cliff,625.250000
6,Bayville,479.428571
12,Glen Head,459.363636


Create Map of North Massapequa

In [24]:
address = 'North Massapequa, NY'

geolocator = Nominatim(user_agent="North_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North Massapequa are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North Massapequa are 40.7004163, -73.4626056.


In [26]:
dfNorthM = df[df['City'] == 'North Massapequa']

In [37]:
# create map of North Massapequa using latitude and longitude values
map = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map with pizza shop locations
for lat, lng in zip(dfNorthM['Lat'], dfNorthM ['Lng']):
    label = folium.Popup('North Massapequa', parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,       parse_html=False).add_to(map)  
    
map

In [45]:
map = folium.Map(location=[40.7004163, -73.4626056], zoom_start=10)

# add markers to map with pizza shop locations
for lat, lng in zip(df['Lat'], df['Lng']):
    label = folium.Popup('Town of Oyster Bay', parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,       parse_html=False).add_to(map)  
    
map
